In [2]:
my_list = [
    '184B5.csv',
'BT483.csv',
'HCC1428.csv',
'HCC1806.csv',
'HCC202.csv',
'Hs578T.csv',
'MCF12A.csv',
'MDAMB231.csv',
'MDAMB468.csv',
'SKBR3.csv',
'UACC3199.csv',
'ZR751.csv',
]

In [3]:
import os, sys 
# from dask_ml.model_selection import train_test_split, RandomizedSearchCV
import dask.dataframe as dd
# from dask_ml.xgboost import XGBRegressor
import joblib
from dask.diagnostics import ProgressBar
# from dask_ml.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold , cross_val_predict , cross_val_score ,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,classification_report, confusion_matrix,accuracy_score,matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
# from dask.distributed import Client

In [4]:
np.random.seed(seed = 101)

In [5]:
df= pd.DataFrame()
# for x in os.listdir("data"):
for x in my_list:
#     print (x)
    df_temp = dd.read_csv("data/%s"%(x) )  #,dtype=dtype)
    df = dd.concat([df,df_temp],axis=0)

In [6]:
df = df.categorize(columns=["time","treatment"])

In [7]:
my_dummies_treatment = dd.get_dummies(df["treatment"])

In [8]:
my_dummies_time = dd.get_dummies(df["time"])

In [9]:
print ("filling NA")
# y = y.compute()
with ProgressBar():
    for m in ["p.HER2","p.PLCg2"] :
#         print (df[m])
        
        df["%s_c"%(m)] =df[m].fillna(df[m].mean() )#, inplace=True )

filling NA


In [10]:
df = df.drop(['p.HER2','p.PLCg2','cell_line','time','treatment','cellID','fileID'],axis=1)

In [11]:
with ProgressBar():
    scaler = StandardScaler()
    scaler.fit(df)
    scaled_data = scaler.transform(df)

[########################################] | 100% Completed | 11.8s
[########################################] | 100% Completed | 12.0s
[########################################] | 100% Completed | 11.7s
[########################################] | 100% Completed | 12.1s


In [13]:
scaled_data = pd.DataFrame(scaled_data,columns=df.columns)  #,index=my_dummies_time.index)

In [14]:
with ProgressBar():
#     scaled_data = scaled_data.compute()
    my_dummies_treatment = my_dummies_treatment.compute()
    my_dummies_time = my_dummies_time.compute()

[########################################] | 100% Completed |  9.1s
[########################################] | 100% Completed |  9.1s


In [15]:
my_dummies_treatment.reset_index(drop=True,inplace=True)
my_dummies_time.reset_index(drop=True,inplace=True)

In [16]:
### concating 
scaled_data = pd.concat([scaled_data,my_dummies_treatment,my_dummies_time],axis=1)

In [17]:
my_dummies_treatment = pd.DataFrame()
my_dummies_time = pd.DataFrame()

In [18]:
with ProgressBar():
    df = df.compute()

[########################################] | 100% Completed | 15.1s


In [ ]:
# genes= ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3','p.MAPKAPK2','p.MEK','p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']

In [ ]:
model = MultiOutputRegressor(RandomForestRegressor(n_jobs=-1,
                                                       random_state=101,
                                                       n_estimators=5000,
                                                       criterion='mae',verbose=2))

In [ ]:
print ("split")
X_train, X_test, y_train, y_test = train_test_split(scaled_data,df,test_size=0.33, random_state=101)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)
print ("R^2:",r2_score(y_test, predictions))
print ("MAE:",mean_absolute_error(y_test, predictions))
print ("MSE:",mean_squared_error(y_test, predictions))

In [ ]:
predictions = model.predict(df)

In [ ]:
print ("############################################")

In [ ]:
print (predictions)